## Libraries


In [1]:
%pip install -r ~/code/benitomartin/FoodScore/requirements.txt

Ignoring tensorflow-macos: markers 'sys_platform == "darwin" and "ARM" in platform_version' don't match your environment
Ignoring tensorflow: markers 'sys_platform != "darwin"' don't match your environment
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanIoU


import pickle

2023-03-22 21:48:35.484477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data import

In [3]:
coord = pd.DataFrame()

for i in range(1, 257, 1):
    path = f"../raw_data/UECFOOD256/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [4]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [5]:
coord.shape

(31645, 6)

In [6]:
coord.head()

,img,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


In [7]:
coord = coord.rename(columns={"img": "img_name"})

In [8]:
coord.head()

,img_name,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


### Rescaling and Normalization

In [9]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD256/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [10]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm
0,1,1,0.0000,0.238333,0.46250,0.810000
1,2,1,0.0250,0.346667,0.72750,0.931667
2,3,1,0.0025,0.183333,0.30375,0.683333
3,4,1,0.0000,0.395000,0.35750,0.893333
4,5,1,0.0100,0.046667,0.95125,0.975000


### add image paths

In [11]:
list_paths = [f"../raw_data/UECFOOD256/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [12]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [13]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.46250,0.810000,../raw_data/UECFOOD256/1/1.jpg
1,2,1,0.0250,0.346667,0.72750,0.931667,../raw_data/UECFOOD256/1/2.jpg
2,3,1,0.0025,0.183333,0.30375,0.683333,../raw_data/UECFOOD256/1/3.jpg
3,4,1,0.0000,0.395000,0.35750,0.893333,../raw_data/UECFOOD256/1/4.jpg
4,5,1,0.0100,0.046667,0.95125,0.975000,../raw_data/UECFOOD256/1/5.jpg


### balancing Dataset

In [14]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [15]:
classes = list(set(rescaled_coord.label))

In [16]:
df = rebalancing(rescaled_coord, classes, av_number= 50, random_state=1)

In [17]:
rescaled_coord[rescaled_coord['label']==100].shape

(104, 7)

In [18]:
df[df['label']==100].shape

(50, 7)

### load downscaled pictures into array

In [19]:
from tqdm.auto import tqdm

/Users/anetacres/.pyenv/versions/3.10.6/envs/FoodScore/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
df.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
21,25,1,0.356250,0.562500,0.701562,1.000000,../raw_data/UECFOOD256/1/25.jpg
46,61,1,0.000000,0.366366,0.456000,0.945946,../raw_data/UECFOOD256/1/61.jpg
51,66,1,0.000000,0.052069,1.000000,0.983979,../raw_data/UECFOOD256/1/66.jpg
54,69,1,0.017817,0.035608,0.959911,0.934718,../raw_data/UECFOOD256/1/69.jpg
64,79,1,0.083750,0.000000,0.910000,0.903333,../raw_data/UECFOOD256/1/79.jpg


In [22]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
14961,67467,104,0.018000,0.045333,0.950000,0.957333,../raw_data/UECFOOD256/104/67467.jpg
24586,265664,192,0.020747,0.127778,0.991701,0.769444,../raw_data/UECFOOD256/192/265664.jpg
4678,2712,28,0.000000,0.000000,1.000000,0.933333,../raw_data/UECFOOD256/28/2712.jpg
16939,78024,123,0.035948,0.068796,0.949346,0.911548,../raw_data/UECFOOD256/123/78024.jpg
10648,7581,71,0.079412,0.176471,0.758824,0.949020,../raw_data/UECFOOD256/71/7581.jpg


In [23]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

100%|██████████| 12800/12800 [02:23<00:00, 89.14it/s] 


In [24]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [25]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [26]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [27]:
len(set(df_shuffled.label))

256

In [28]:
tvImages, testImages,tvLabels, testLabels,tvBBoxes, testBBoxes,tvPaths, testPaths=\
train_test_split(images,
                 labels,
                 bboxes,
                 paths,
                 test_size=0.20,
                 random_state=42)

In [29]:
trainImages, valImages,trainLabels, valLabels,trainBBoxes, valBBoxes, trainPaths, valPaths=\
train_test_split(tvImages,
                 tvLabels,
                 tvBBoxes,
                 tvPaths,
                 test_size=0.20,
                 random_state=42)

## Model Inceptionv3

In [30]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [31]:
inception_v3 = InceptionV3(weights="imagenet",
                        include_top=False,
                        input_tensor=layers.Input((224, 224, 3)))


inception_v3.trainable = False

flatten = inception_v3.output
flatten = layers.Flatten()(flatten)

bboxHead = layers.Dense(128, activation="relu")(flatten)
bboxHead = layers.Dense(64, activation="relu")(bboxHead)
bboxHead = layers.Dense(32, activation="relu")(bboxHead)
bboxHead = layers.Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)

softmaxHead = layers.Dense(512, activation="relu")(flatten)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(512, activation="relu")(softmaxHead)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(len(set(df_shuffled.label)), activation="softmax", name="class_label")(softmaxHead)

2023-03-22 22:23:53.968951: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


87910968/87910968 [==============================] - 5s 0us/step


In [32]:
model_inception = Model(
    inputs=inception_v3.input,
    outputs=(bboxHead, softmaxHead))

In [33]:
losses = {
    "class_label": 'categorical_crossentropy',
    "bounding_box": "mse"
}

In [34]:
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

In [42]:
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}

In [35]:
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

In [36]:
valTargets = {
    "class_label": valLabels,
    "bounding_box": valBBoxes
}

In [37]:
metrics = {
    "class_label": "categorical_accuracy",
    "bounding_box": MeanIoU(num_classes=len(set(df_shuffled.label)))
}

In [38]:
opt = Adam(0.001)

In [39]:
model_inception.compile(loss=losses, 
              optimizer=opt, 
              metrics=metrics, 
              loss_weights=lossWeights)

print(model_inception.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [40]:
es = EarlyStopping(monitor = 'categorical_accuracy',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [44]:
history_inception = model_inception.fit(
    trainImages, 
    trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=32,
    epochs=100,
    callbacks = [es],
    verbose=1
)

Epoch 1/100
256/256 [==============================] - 1973s 8s/step - loss: 5.5823 - bounding_box_loss: 0.0207 - class_label_loss: 5.5616 - bounding_box_mean_io_u: 0.3229 - class_label_categorical_accuracy: 0.0052 - val_loss: 5.5740 - val_bounding_box_loss: 0.0204 - val_class_label_loss: 5.5537 - val_bounding_box_mean_io_u: 0.3243 - val_class_label_categorical_accuracy: 0.0020
Epoch 2/100
256/256 [==============================] - 988s 4s/step - loss: 5.5728 - bounding_box_loss: 0.0207 - class_label_loss: 5.5521 - bounding_box_mean_io_u: 0.3229 - class_label_categorical_accuracy: 0.0044 - val_loss: 5.5763 - val_bounding_box_loss: 0.0204 - val_class_label_loss: 5.5559 - val_bounding_box_mean_io_u: 0.3243 - val_class_label_categorical_accuracy: 0.0020
Epoch 3/100
256/256 [==============================] - 25932s 102s/step - loss: 5.5669 - bounding_box_loss: 0.0207 - class_label_loss: 5.5462 - bounding_box_mean_io_u: 0.3229 - class_label_categorical_accuracy: 0.0046 - val_loss: 5.5786 - 

KeyboardInterrupt: 

## Save model

In [45]:
# Inceptionv3

pickle.dump(model_inception, open('inception_256classes.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://73fb8b88-484f-4c12-9c6f-d5344e14abb6/assets


INFO:tensorflow:Assets written to: ram://73fb8b88-484f-4c12-9c6f-d5344e14abb6/assets
